# 데이터 분석 프로젝트

## 매출

In [2]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

## 데이터 로드 및 전처리

In [4]:
# 데이터를 로드합니다.
data_path = './data_in/total_df.csv'
data = pd.read_csv(data_path)

# 결측치를 제거합니다.
data = data.dropna()

# 날짜 관련 컬럼을 datetime 형식으로 변환합니다.
date_columns = ['Order_purchase_timestamp', 'Order_delivered_carrier_date', 'Order_delivered_customer_date', 'Order_estimated_delivery_date', 'Review_creation_date', 'Review_answer_timestamp']
for col in date_columns:
    data[col] = pd.to_datetime(data[col])

C:\Users\USER\AppData\Local\Temp\ipykernel_1264\2974882245.py:3: DtypeWarning: Columns (16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


## 파생 변수 생성

In [5]:
# 파생 변수를 생성합니다.
data['Delivery_speed'] = (data['Order_delivered_customer_date'] - data['Order_purchase_timestamp']).dt.days
data['Review_response_time'] = (data['Review_answer_timestamp'] - data['Review_creation_date']).dt.days
customer_order_count = data['Customer_unique_id'].value_counts().to_dict()
data['Customer_order_count'] = data['Customer_unique_id'].apply(lambda x: customer_order_count.get(x, 0))

## 회귀 모델 구축 및 평가

In [6]:
# 특성과 타겟 변수를 선택합니다.
X = data[['Price', 'Freight_value', 'Review_score', 'Delivery_speed', 'Review_response_time', 'Customer_order_count']]
y = data['Payment_value']

# 데이터를 훈련 세트와 테스트 세트로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

# 선형 회귀 모델을 생성하고 훈련합니다.
model = LinearRegression()
model.fit(X_train, y_train)

# 예측을 수행하고 모델을 평가합니다.
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 회귀 계수를 출력합니다.
model.coef_

array([ 1.05732816e+00,  1.44479208e+00, -1.68870670e+01, -7.29594800e-01,
        9.23124910e-03,  9.64743428e+00])

In [7]:

# 데이터 정규화
from sklearn.preprocessing import StandardScaler

# 정규화를 위한 스케일러 초기화
scaler = StandardScaler()

# 훈련 데이터를 정규화
X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터를 정규화 (훈련 데이터로 학습된 스케일러 사용)
X_test_scaled = scaler.transform(X_test)

# 정규화된 데이터로 모델 재훈련
model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, y_train)

# 정규화된 데이터로 예측 및 성능 평가
y_pred_scaled = model_scaled.predict(X_test_scaled)
mse_scaled = mean_squared_error(y_test, y_pred_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)

# 결과 출력
print('Normalized MSE:', mse_scaled)
print('Normalized R^2 Score:', r2_scaled)
print('Normalized Coefficients:', model_scaled.coef_)


Normalized MSE: 20747.148867545522
Normalized R^2 Score: 0.6348421660866903
Normalized Coefficients: [ 1.90708273e+02  2.21485784e+01 -2.29274435e+01 -6.98588688e+00
  9.24587588e-02  2.86088765e+01]


## 결과 해석

In [8]:
# 모델 성능과 계수를 출력합니다.
print('MSE:', mse)
print('R^2 Score:', r2)
print('Coefficients:', model.coef_)

MSE: 20747.148867545464
R^2 Score: 0.6348421660866913
Coefficients: [ 1.05732816e+00  1.44479208e+00 -1.68870670e+01 -7.29594800e-01
  9.23124910e-03  9.64743428e+00]
